In [ ]:
#getting all the packages
import os
import sys

print(f'os.environ={os.environ}')
print(f'sys.argv={sys.argv}')
print(f"MLFLOW_TRACKING_URI{os.environ.get('MLFLOW_TRACKING_URI')}")
print(f"sys.executable={sys.executable}")
print(f"sys.path={sys.path}")

os.environ=environ({'NV_LIBCUBLAS_DEV_VERSION': '11.4.1.1043-1', 'NV_CUDA_COMPAT_PACKAGE': 'cuda-compat-11-2', 'NV_CUDNN_PACKAGE_DEV': 'libcudnn8-dev=8.1.1.33-1+cuda11.2', 'LD_LIBRARY_PATH': '/usr/local/nvidia/lib:/usr/local/nvidia/lib64', 'NV_LIBNCCL_DEV_PACKAGE': 'libnccl-dev=2.8.4-1+cuda11.2', 'TCLLIBPATH': '/usr/share/tcltk/tcllib1.19', 'CLOUDSDK_PYTHON': 'python3', 'LANG': 'en_US.UTF-8', 'NV_LIBNPP_DEV_PACKAGE': 'libnpp-dev-11-2=11.3.2.152-1', 'HOSTNAME': '5b1ede0dbb00', 'OLDPWD': '/', 'CLOUDSDK_CONFIG': '/content/.config', 'NV_LIBNPP_VERSION': '11.3.2.152-1', 'NV_NVPROF_DEV_PACKAGE': 'cuda-nvprof-11-2=11.2.152-1', 'NVIDIA_VISIBLE_DEVICES': 'all', 'NV_NVPROF_VERSION': '11.2.152-1', 'NV_LIBCUSPARSE_VERSION': '11.4.1.1152-1', 'DATALAB_SETTINGS_OVERRIDES': '{"kernelManagerProxyPort":6000,"kernelManagerProxyHost":"172.28.0.3","jupyterArgs":["--ip=172.28.0.2"],"debugAdapterMultiplexerPath":"/usr/local/bin/dap_multiplexer","enableLsp":true}', 'NV_LIBCUBLAS_DEV_PACKAGE': 'libcublas-dev-1

In [ ]:
from google.colab import drive 
drive.mount('/content/gdrive/')

Mounted at /content/gdrive/


In [ ]:
import os
! git clone https://github.com/HelenGuohx/logbert.git


Cloning into 'logbert'...
remote: Enumerating objects: 130, done.
remote: Counting objects: 100% (40/40), done.
remote: Compressing objects: 100% (35/35), done.
remote: Total 130 (delta 9), reused 5 (delta 5), pack-reused 90
Receiving objects: 100% (130/130), 210.44 KiB | 3.09 MiB/s, done.
Resolving deltas: 100% (21/21), done.


In [ ]:
import sys
sys.path.append('/content/logbert/')
from loglizer.models import InvariantsMiner, PCA, IsolationForest, OneClassSVM, LogClustering, LR, SVM
from loglizer import dataloader, preprocessing
from loglizer.utils import metrics
from logdeep.dataset.session import sliding_window

In [ ]:
#%cd logbert
import os
import gc
import pandas as pd
import numpy as np
import argparse
from tqdm import tqdm
import argparse
import numpy as np
import random
from sklearn.model_selection import GridSearchCV
from sklearn import svm
from sklearn.utils import shuffle

In [ ]:
#train test split 
ouput_dir = "/content/gdrive/MyDrive/bgl/"
middle_dir = ""
log_file = "bgl2"

(x_train, y_train), (x_test, y_test) = dataloader.load_data(ouput_dir, middle_dir, log_file, is_mapping=True)
feature_extractor = preprocessing.FeatureExtractor()
x_train = feature_extractor.fit_transform(x_train)
x_test = feature_extractor.transform(x_test)

Train normal size: 13527
Train abnormal size: 1398
Total logkey(exclude 0:UNK) 1000
Test normal size: 20291
Test abnormal size: 2099
num_unk_event in test data: 0
====== Transformed train data summary ======
Train data shape: 14925-by-830

====== Transformed test data summary ======
Test data shape: 22390-by-830



In [ ]:
x_train[3]

array([0., 0., 0., 0., 1., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0.

In [ ]:
#PCA analysis
%%time
print("="*20 + " Model: PCA " + "="*20)
for th in np.arange(1):
    print("theshold", th)
    model = PCA(n_components=0.8, threshold=1, c_alpha = 1.9600)
    model.fit(x_train)
    print('Train validation:')
    precision, recall, f1 = model.evaluate(x_train, y_train)
    print('Test validation:')
    precision, recall, f1 = model.evaluate(x_test, y_test)

==================== Model: PCA ====================
theshold 0
====== Model summary ======
n_components: 4
Project matrix shape: 830-by-830
SPE threshold: 1

Train validation:
====== Evaluation summary ======
Confusion Matrix: TP: 1325, FP: 11730, TN: 1797, FN: 73
Precision: 10.149%, recall: 94.778%, F1-measure: 18.335%

Test validation:
====== Evaluation summary ======
Confusion Matrix: TP: 1971, FP: 17700, TN: 2591, FN: 128
Precision: 10.020%, recall: 93.902%, F1-measure: 18.108%

CPU times: user 18.8 s, sys: 2.93 s, total: 21.8 s
Wall time: 12.6 s


In [ ]:
#isolation forest
%%time
print("="*20 + " Model: IsolationForest " + "="*20)
model = IsolationForest(n_estimators=100, max_samples='auto', contamination='auto', random_state=19)
model.fit(x_train)
print('Train validation:')
precision, recall, f1 = model.evaluate(x_train, y_train)
print('Test validation:')
precision, recall, f1 = model.evaluate(x_test, y_test)

==================== Model: IsolationForest ====================
====== Model summary ======
Train validation:
====== Evaluation summary ======
Confusion Matrix: TP: 183, FP: 0, TN: 13527, FN: 1215
Precision: 100.000, recall: 13.090, F1-measure: 23.150

Test validation:
====== Evaluation summary ======
Confusion Matrix: TP: 292, FP: 0, TN: 20291, FN: 1807
Precision: 100.000, recall: 13.911, F1-measure: 24.425

CPU times: user 21.2 s, sys: 1.05 s, total: 22.3 s
Wall time: 22.3 s


In [ ]:
#SVM
%%time
print("="*20 + " Model: one class SVM " + "="*20)
model = OneClassSVM(kernel='rbf')
model.fit(x_train, y_train)

print('Train validation:')
precision, recall, f1 = model.evaluate(x_train, y_train)
print('Test validation:')
precision, recall, f1 = model.evaluate(x_test, y_test)

==================== Model: one class SVM ====================
====== Model summary ======
Train validation:
====== Evaluation summary ======
Confusion Matrix: TP: 214, FP: 13527, TN: 0, FN: 1184
Precision: 1.557, recall: 15.308, F1-measure: 2.827

Test validation:
====== Evaluation summary ======
Confusion Matrix: TP: 324, FP: 20291, TN: 0, FN: 1775
Precision: 1.572, recall: 15.436, F1-measure: 2.853

CPU times: user 6min 55s, sys: 435 ms, total: 6min 56s
Wall time: 6min 55s


In [ ]:
#Clustering
%%time
print("="*20 + " Model: LogClustering " + "="*20)
max_dist = 0.3  # the threshold to stop the clustering process
anomaly_threshold = 0.3  # the threshold for anomaly detection
model = LogClustering(max_dist=max_dist, anomaly_threshold=anomaly_threshold)
model.fit(x_train[y_train == 0, :])  # Use only normal samples for training
print('Train validation:')
precision, recall, f1 = model.evaluate(x_train, y_train)
print('Test validation:')
precision, recall, f1 = model.evaluate(x_test, y_test)

==================== Model: LogClustering ====================
====== Model summary ======
Starting offline clustering...
Processed 1000 instances.
Found 86 clusters offline.

Starting online clustering...
Processed 2000 instances.
Processed 4000 instances.
Processed 6000 instances.
Processed 8000 instances.
Processed 10000 instances.
Processed 12000 instances.
Processed 13527 instances.
Found 166 clusters online.

Train validation:
====== Evaluation summary ======
Confusion Matrix: TP: 847, FP: 8, TN: 13519, FN: 551
Precision: 99.064, recall: 60.587, F1-measure: 75.189

Test validation:
====== Evaluation summary ======
Confusion Matrix: TP: 1269, FP: 56, TN: 20235, FN: 830
Precision: 95.774, recall: 60.457, F1-measure: 74.124

CPU times: user 2min 30s, sys: 1.59 s, total: 2min 31s
Wall time: 2min 31s
